In [64]:
from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import matplotlib.pyplot as plt 
import matplotlib.dates as dates
import ipywidgets as widgets


register_matplotlib_converters()
%matplotlib inline

In [65]:
df = None
last = datetime.now() - timedelta(hours=1)


def get_data():
    """
    Get the latest data. Just return the cached copy
    if less than one hour has elapsed.
    """
    global last, df
    now = datetime.now()
    if now - last < timedelta(hours=1):
        return df

    last = now    
    start = date(2020, 1, 22)
    end = date.today()

    df = pd.DataFrame()
    while start <= end:
        try:
            df_day = pd.read_csv(f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{start.month:02d}-{start.day:02d}-{start.year}.csv')
            df_day['Date'] = start
            df = df.append(df_day)
        except Exception:
            pass
        start += timedelta(days=1)
    
    del df['Last Update']
    df = df.replace({'Country/Region': {'Mainland China': 'China'}})
    df.fillna({'Province/State': ''}, inplace=True)
    for col in ['Confirmed', 'Deaths', 'Recovered']:
        df.fillna({col: 0}, inplace=True)
    return df

In [66]:
df = get_data()
df.tail(5)

,Confirmed,Country/Region,Date,Deaths,Latitude,Longitude,Province/State,Recovered
279,0.0,Guernsey,2020-03-18,0.0,49.4500,-2.580,,0.0
280,0.0,Jersey,2020-03-18,0.0,49.1900,-2.110,,0.0
281,0.0,Puerto Rico,2020-03-18,0.0,18.2000,-66.500,,0.0
282,0.0,Republic of the Congo,2020-03-18,0.0,-1.4400,15.556,,0.0
283,0.0,The Gambia,2020-03-18,0.0,13.4667,-16.600,,0.0


In [99]:
def aggregate_for_locations(df, locations, sub_locs=None, do_diffs=True, fields=None):
    if fields is None:
        fields = ['Confirmed', 'Deaths', 'Recovered']
    cols = ['Date']
    cols.extend(fields)
    
    if isinstance(locations, str):
        locations = [locations]
        
    if sub_locs:
        in_loc = df[(df['Country/Region'].isin(locations)) & (df['Province/State'].isin(sub_locs))]
    else:
        in_loc = df[df['Country/Region'].isin(locations)]
        
    result = in_loc[cols].groupby('Date').sum()
    
    if do_diffs:
        for f in fields:
            result[f] = result[f].diff()
            result.fillna({f: 0}, inplace=True)
    return result


In [68]:
def plot_time_series(df, title):
    fig, ax = plt.subplots()
    ax.plot_date(df.index, df, 'v-')
    ax.xaxis.set_minor_locator(dates.WeekdayLocator(byweekday=(1), interval=1))
    ax.xaxis.set_minor_formatter(dates.DateFormatter('%d\n%a'))
    ax.xaxis.grid(True, which="minor")
    ax.yaxis.grid()
    ax.xaxis.set_major_locator(dates.MonthLocator())
    ax.xaxis.set_major_formatter(dates.DateFormatter('\n\n\n%b\n%Y'))
    ax.legend(df.columns, loc='upper left', shadow=True)
    ax.set_title(title)
    plt.tight_layout()
    return plt

In [69]:
locations = sorted(list(df['Country/Region'].unique()))

In [122]:
states = {
    'AL': ['Alabama'],
    'AK': ['Alaska'],
    'AZ': ['Arizona'],
    'AR': ['Arkansas'],
    'CA': ['California'],
    'CO': ['Colorado'],
    'CT': ['Connecticut'],
    'D.C.': ['District of Columbia'],
    'DE': ['Delaware'],
    'FL': ['Florida'],
    'GA': ['Georgia'],
    'HI': ['Hawaii'],
    'ID': ['Idaho'],
    'IL': ['Illinois'],
    'IN': ['Indiana'],
    'IA': ['Iowa'],
    'KS': ['Kansas'],
    'KY': ['Kentucky'],
    'LA': ['Louisiana'],
    'ME': ['Maine'],
    'MD': ['Maryland'],
    'MA': ['Massachusetts'],
    'MI': ['Michigan'],
    'MN': ['Minnesota'],
    'MS': ['Mississippi'],
    'MO': ['Missouri'],
    'MT': ['Montana'],
    'NE': ['Nebraska'],
    'NV': ['Nevada'],
    'NH': ['New Hampshire'],
    'NJ': ['New Jersey'],
    'NM': ['New Mexico'],
    'NY': ['New York'],
    'NC': ['North Carolina'],
    'ND': ['North Dakota'],
    'OH': ['Ohio'],
    'OK': ['Oklahoma'],
    'OR': ['Oregon'],
    'PA': ['Pennsylvania'],
    'PR': ['Puerto Rico'],
    'RI': ['Rhode Island'],
    'SC': ['South Carolina'],
    'SD': ['South Dakota'],
    'TN': ['Tennessee'],
    'TX': ['Texas'],
    'UT': ['Utah'],
    'VT': ['Vermont'],
    'VA': ['Virginia'],
    'WA': ['Washington'],
    'WV': ['West Virginia'],
    'WI': ['Wisconsin'],
    'WY': ['Wyoming'],
    'U.S.': [] # other territories
}

# Extend the above with all the counties/cities that were broken out separately
for s in df[df['Country/Region'] == 'US']['Province/State'].unique():
    i = s.find(',')
    if i < 0:
        continue
    st = s[i+2:i+5].strip()
    if st not in states:
        st += '.'
        if st not in states:
            print(f'Failed with {s}/{st}')
            continue
    states[st].append(s)


In [70]:
picker = widgets.SelectMultiple(
                        options=locations,
                        value=['US'],
                        description='Locations'
                )

In [96]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location, do_diffs=False), 
                                                   title=f'{" ".join(Location)}: Cumulative'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(181,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [97]:
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, Location), 
                                                   title=f'{" ".join(Location)}: New'),
                 Location=picker)

interactive(children=(SelectMultiple(description='Locations', index=(181,), options=(' Azerbaijan', 'Afghanist…

<function __main__.<lambda>(Location)>

In [126]:
picker2 = widgets.Select(
                        options=states.keys(),
                        value='WA',
                        description='States'
                )
widgets.interact(lambda Location: plot_time_series(aggregate_for_locations(df, 'US', sub_locs=states[Location], do_diffs=False), 
                                                   title=f'{Location}: Cumulative'),
                 Location=picker2)

interactive(children=(Select(description='States', index=48, options=('AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT'…

<function __main__.<lambda>(Location)>